### SQLite: Country Club Case Study

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [15]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [16]:

database = "sqlite_db_pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks(conn)

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [17]:
with conn:
    cur= conn.cursor()
    query10= ''' SELECT f.name, SUM(
CASE WHEN b.memid =0
THEN (
slots * guestcost
)
ELSE (
slots * membercost
)
END ) AS revenue
FROM Bookings AS b
INNER JOIN Facilities AS f
USING ( facid )
GROUP BY f.name
HAVING revenue <1000
ORDER BY revenue
LIMIT 0 , 30'''
    rs10= cur.execute(query10)
    df10= pd.DataFrame(rs10.fetchall())
    df10.columns= [i[0] for i in rs10.description]
    print(df10)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [20]:
with conn:
    cur= conn.cursor()
    query11= ''' SELECT ( m.firstname || ' ' || m.surname ) AS member,
CASE WHEN r.recommendedby IS NULL
THEN 'no reccomendation'
ELSE ( r.firstname || ' ' || r.surname )
END AS recommended_by
FROM Members AS m
LEFT JOIN Members AS r ON m.memid = r.recommendedby
WHERE m.memid !=0
ORDER BY member'''
    rs11= cur.execute(query11)
    df11= pd.DataFrame(rs11.fetchall())
    df11.columns= [i[0] for i in rs11.description]
    print(df11)

                     member           recommended_by
0            Anna Mackenzie        no reccomendation
1                Anne Baker        no reccomendation
2              Burton Tracy          Ponder Stibbons
3              Charles Owen        no reccomendation
4              Darren Smith          Janice Joplette
5              Darren Smith           Gerald Butters
6              Darren Smith             Charles Owen
7              Darren Smith               Jack Smith
8              Darren Smith           Anna Mackenzie
9              Darren Smith        no reccomendation
10            David Farrell        no reccomendation
11              David Jones            Douglas Jones
12             David Pinker        no reccomendation
13            Douglas Jones        no reccomendation
14            Erica Crumpet        no reccomendation
15           Florence Bader         Ramnaresh Sarwin
16           Gerald Butters          Matthew Genting
17         Henrietta Rumney        no reccomen

### Q12: Find the facilities with their usage by member, but not guests 

In [22]:
with conn:
    cur= conn.cursor()
    query12= '''SELECT f.name, COUNT( b.bookid ) AS member_bookings
FROM Facilities AS f
INNER JOIN Bookings AS b
USING ( facid )
WHERE b.memid !=0
GROUP BY f.name'''
    rs12= cur.execute(query12)
    df12= pd.DataFrame(rs12.fetchall())
    df12.columns= [i[0] for i in rs12.description]
    print(df12)

              name  member_bookings
0  Badminton Court              344
1   Massage Room 1              421
2   Massage Room 2               27
3       Pool Table              783
4    Snooker Table              421
5     Squash Court              195
6     Table Tennis              385
7   Tennis Court 1              308
8   Tennis Court 2              276


### * Q13: Find the facilities usage by month, but not guests 

In [26]:
with conn:
    cur= conn.cursor()
    query13= '''SELECT f.name
              ,strftime('%m', b.starttime) AS month
              ,count(b.bookid) AS member_bookings
          FROM Facilities as f
                 INNER JOIN
               Bookings as b
            using(facid) 
         WHERE b.memid <> 0
         GROUP BY month,f.name
         ORDER BY f.name,month'''
    rs13= cur.execute(query13)
    df13= pd.DataFrame(rs13.fetchall())
    df13.columns= [i[0] for i in rs13.description]
    print(df13)

               name month  member_bookings
0   Badminton Court    07               51
1   Badminton Court    08              132
2   Badminton Court    09              161
3    Massage Room 1    07               77
4    Massage Room 1    08              153
5    Massage Room 1    09              191
6    Massage Room 2    07                4
7    Massage Room 2    08                9
8    Massage Room 2    09               14
9        Pool Table    07              103
10       Pool Table    08              272
11       Pool Table    09              408
12    Snooker Table    07               68
13    Snooker Table    08              154
14    Snooker Table    09              199
15     Squash Court    07               23
16     Squash Court    08               85
17     Squash Court    09               87
18     Table Tennis    07               48
19     Table Tennis    08              143
20     Table Tennis    09              194
21   Tennis Court 1    07               65
22   Tennis